# Methods

The behaviour of a function depends on the type of its input arguments. In other words, it is possible to define several functions, called *methods*, with the same name but with different input arguments. When a function is invoked, Julia decides which method to call on the basis of the function's input arguments, a mechanism known as *multiple dispatch*.

## Follow-up example using user-defined shape types

In [ ]:
abstract Shape
abstract ConicSection <: Shape

In [ ]:
type Point{T<:Real}
  x::T
  y::T
end

In [ ]:
type Circle{T<:Real} <: ConicSection
  centre::Point{T} # Circle center is of type Point{T}
  radius::T # Circle radius is of type T
end

In [ ]:
# Define Rectangle to be a sub-type of the abstract Shape type
type Rectangle{T<:Real} <: Shape
  ll::Point{T} # Lower left vertex of rectangle is of type Point{T}
  ur::Point{T} # Upper right vertex of rectangle is of type Point{T}
end

In [ ]:
# Construct a circle with centre (0.0, 0.0) and radius 2
circle = Circle(Point(0.0, 0.0), 2.0)

In [ ]:
# Construct a rectangle with lower left vertex (0.0, 0.0) and upper right vertex (2.0, 1.0)
rectangle = Rectangle(Point(0.0, 0.0), Point(2.0, 1.0))

## Multiple dispatch

In [ ]:
# Define the area() function for Circle arguments, giving the circle's area
area(shape::Circle) = pi*abs2(shape.radius)

In [ ]:
area(circle)

In [ ]:
# Define the area() function for Rectangle arguments
# The function name area() is the same for different input arguments
# This is called multiple dispatch
area(shape::Rectangle) = abs(shape.ll.y-shape.ur.y)*abs(shape.ll.x-shape.ur.x)

In [ ]:
area(rectangle)

## Parametric functions and type inference on input arguments

In [ ]:
# resize() function returns a new circle with the same centre and radius rescaled by some coefficient c 
resize(shape::Circle, c) = Circle(shape.centre, c*shape.radius)

In [ ]:
# Resize circle by halving its radius
resize(circle, 0.5)

In [ ]:
# The type of c was previously omitted
# It is now stated explicitly that c is of type Float64
resize(shape::Circle, c::Float64) = Circle(shape.centre, c*shape.radius)

In [ ]:
# Resize circle by halving its radius using the redefined resize function
resize(circle, 0.5)

In [ ]:
# Define a resize() method in which c is of type Int64
resize(shape::Circle, c::Int16) = Circle(shape.centre, 10*c*shape.radius)

In [ ]:
# Multiple dispatch picks the resize() method to call depending on the type of c
resize(circle, 1.), resize(circle, 1)

In [ ]:
# Now resize is a parametric function with type parameter T shared between its input arguments
resize{T}(shape::Circle{T}, c::T) = Circle(shape.centre, c*shape.radius)

In [ ]:
# Resize circle by halving its radius using the redefined resize function
resize(circle, 1.)

## Optional arguments

In [ ]:
# Provide the scaling factor c as an optional argument
# The default value of c is set to return a circle of radius 1
resize(shape::Circle, c=1/shape.radius) = Circle(shape.centre, c*shape.radius)

In [ ]:
# Call resize with one argument (the second argument takes its default value)
resize(circle)

## Keyword arguments

In [ ]:
# Provide the scaling factor as the named argument scaling
# The default value of scaling is set to return a circle of radius 1
resize(shape::Circle; scaling=1/shape.radius) = Circle(shape.centre, scaling*shape.radius)

In [ ]:
# Call resize with scaling=2
resize(circle, scaling=2)

## Functions changing their input

In [ ]:
# resize! is defined in Base already, so import it to extend it
import Base.resize!

# resize function returns a new circle with the same centre and radius rescaled by some coefficient c
# Names of functions changing their input are conventionally suffixed by "!"
function resize!(shape::Circle, c)
  shape.radius = c*shape.radius
  shape
end

In [ ]:
resize!(circle, 0.5)

circle

## Inner constructors

In [ ]:
# It is possible to call the default Circle constructor with a negative radius
Circle(Point(0.0, 0.0), -1.0)

In [ ]:
# This is one way of prohibiting negative radius
type CircleWithCheck{T<:Real} <: ConicSection
  centre::Point{T} # Circle center is of type Point{T}
  radius::T # Circle radius is of type T
    
  function CircleWithCheck(centre, radius)
    radius > zero(radius) || error("Circle radius must be positive.")
    new(centre, radius)
  end
end

In [ ]:
# Outer constructor for CircleWithCheck
CircleWithCheck{T<:Real}(centre::Point{T}, radius::T) = CircleWithCheck{T}(centre, radius)

In [ ]:
# Passing a negative radius to the new constructor throws an error
CircleWithCheck(Point(0.0, 0.0), -1.0)

## Outer constructors

In [ ]:
# Constructors can be provided outside the type definition
# The following constructor facilitates defining a unit circle with fewer arguments
Circle() = Circle(Point(0.0, 0.0), 1.0)

In [ ]:
# Call the outer constructor to define a unit circle
Circle()